In [1]:
import snowflake.connector
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100
plt.rcParams["figure.figsize"] = (10,8)
plt.style.use('seaborn-darkgrid')

In [4]:
con = snowflake.connector.connect(
    user="tliang@endpointclosing.com",
    account="endpoint",
    authenticator="externalbrowser",
    role="SNOWFLAKE_DATA_ENGINEERING",
    warehouse="DATAENGINEERING_WH"
)
cur = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


### load the datasets

In [8]:
# lead
query = "SELECT * FROM LAKE.SALESFORCE.LEAD WHERE IS_DELETED = 'FALSE' AND STATUS != 'New'"
all_lead = cur.execute(query).fetch_pandas_all().reset_index()

In [6]:
states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AS': 'American Samoa',#
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'GU': 'Guam',#
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MP': 'Northern Mariana Islands',#
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'PR': 'Puerto Rico',#
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VI': 'Virgin Islands',#
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

def state_std(x:str):

    if x is None:
        return ""

    elif (len(x) == 2) & (x.isalpha()):
        return x.upper()
    else:
        for key, value in states.items():
            if x.lower() == value.lower():
                return key
        return ""


In [9]:
all_lead["STATE_STD"] = all_lead["STATE"].apply(state_std)

In [10]:
# task
query = "SELECT * FROM LAKE.SALESFORCE.TASK WHERE WHO_ID IN (SELECT ID FROM LAKE.SALESFORCE.LEAD WHERE IS_DELETED = 'FALSE' AND STATUS != 'New')"
all_task = cur.execute(query).fetch_pandas_all().reset_index()

In [11]:
# lead_hist
query = "SELECT * FROM LAKE.SALESFORCE.LEAD_HISTORY WHERE LEAD_ID IN (SELECT ID FROM LAKE.SALESFORCE.LEAD WHERE IS_DELETED = 'FALSE' AND STATUS != 'New')"
all_lead_hist = cur.execute(query).fetch_pandas_all().reset_index()

### rollback data using cutoff date

#### rollback_wa_lead

In [12]:
cutoff_date_start = dt.date(2021,7,1)
cutoff_date_end = dt.date(2022,1,1)

In [13]:
# leadConvertted, created, leadMerged:  field has ineffective value changes (None to None) so that it's excluded
# Owner: Owner can't be found in lead columns
# ownerAssignment: need a extra filter to specify DATATYPE ==EntityId

# function to standardize the field name
def field_std(x:str) -> str:

    field_replace_dict = {"MobilePhone":"MOBILE_PHONE",
                    "LeadSource":"LEAD_SOURCE", 
                    "Brokerage1__c": "BROKERAGE_1_C",
                    "ownerAssignment": "OWNER_ID",
                    "HasOptedOutOfEmail":"HAS_OPTED_OUT_OF_EMAIL"}

    for key in field_replace_dict.keys():
        if x == key:
            x = field_replace_dict[key]
            
    x = x.upper()
    x = x.replace("__","_")
    
    return x

In [16]:
# rollback function
# to rollback the status of the lead status, using the lead_hist, changing the value at the 'lead' dataset

# steps:
# set the cutoff date: cutoff date start, cutoff date end
# forward search the lead_hist, using lead_id and the lead_hist in range cutoff date, find the earlist change of each type of changes, and extract the old_value, replace the value on related columns
# return the dataset

def rollback_lead(input_lead_df:pd.DataFrame, 
                    lead_hist_df:pd.DataFrame, 
                    cutoff_date_start:dt.date, 
                    cutoff_date_end:dt.date) -> pd.DataFrame:
    
    #subset the data using lead id and cutoff dates
    subset_hist_df = lead_hist_df[(lead_hist_df["LEAD_ID"].isin(input_lead_df["ID"])) & (lead_hist_df["CREATED_DATE"].dt.date >= cutoff_date_start) & (lead_hist_df["CREATED_DATE"].dt.date <= cutoff_date_end)]

    # drop rows that contains fields which does not refer back to lead dataset
    filter_values = ["leadConvertted","created","leadMerged","Owner"]
    subset_hist_df = subset_hist_df[~subset_hist_df["FIELD"].isin(filter_values)]
    
    # ownerAssignment: need a extra filter to drop DATATYPE ==EntityId
    subset_hist_df = subset_hist_df[~(((subset_hist_df["FIELD"] == "Owner") | (subset_hist_df["FIELD"] == "ownerAssignment") )& (subset_hist_df["DATA_TYPE"] == "EntityId"))]

    # apply the field standardization function
    subset_hist_df["FIELD_STD"] = subset_hist_df["FIELD"].apply(field_std)

    # create a dataframe for output
    output_df = pd.DataFrame()

    # need to check cases with same fields
    for id in input_lead_df["ID"]:
        # extract the lead hist of the certain id
        id_subset = subset_hist_df[subset_hist_df["LEAD_ID"] == id]
        # filter only the first changes, by kind of fields
        id_subset = id_subset.loc[id_subset.groupby("FIELD_STD")["CREATED_DATE"].idxmin()]
        # create the dictionary to replace
        replace_dict = dict(zip(id_subset["FIELD_STD"],id_subset["OLD_VALUE"]))
        # pullout the id realted row for replacement
        lead_id_row = input_lead_df[input_lead_df["ID"] == id]
        # extract id index
        id_idx = lead_id_row.index
        # convert id row to dictonary
        lead_id_dict = lead_id_row.to_dict("index")[id_idx.values[0]]
        # use replace dict to update
        lead_id_dict.update(replace_dict)
        # transform back to dataframe and append to output dataframe
        output_df = pd.concat([output_df, pd.DataFrame(lead_id_dict, index = id_idx)])

    return output_df

In [17]:
cutoff_all_lead = rollback_lead(all_lead,all_lead_hist,cutoff_date_start,cutoff_date_end)

#### cutoff_all_task

In [ ]:
cutoff_all_task= all_task[pd.to_datetime(all_task["ACTIVITY_DATE"]).dt.date < cutoff_date_end]

#### cutoff_wa_all_hist

In [ ]:
cutoff_all_lead_hist = all_lead_hist[all_lead_hist["CREATED_DATE"].dt.date < cutoff_date_end]

#### cutoff_wa_lead_listing

In [28]:
cutoff_all_lead_listing = all_lead_listing[pd.to_datetime(all_lead_listing["ACTIVE_DATE_C"]).dt.date < cutoff_date_end]

### output the data

In [43]:
# cutoff_all_lead.to_csv("../data/cutoff_all_lead.csv")

In [44]:
# cutoff_all_task.to_csv("../data/cutoff_all_task.csv")

In [47]:
# cutoff_all_lead_hist.to_csv("../data/cutoff_all_lead_hist.csv")

In [46]:
# cutoff_all_lead_listing.to_csv("../data/cutoff_all_lead_listing.csv")

### Census ACS

#### DP02

In [48]:
social_char = pd.read_csv("../data/ACSDP5Y2020.DP02_data_with_overlays_2022-03-17T153342.csv", header= 1, low_memory=False)

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/2469074121.py:1: DtypeWarning: Columns (349,493) have mixed types. Specify dtype option on import or set low_memory=False.
  social_char = pd.read_csv("../data/ACSDP5Y2020.DP02_data_with_overlays_2022-03-17T153342.csv", header= 1)


In [83]:
social_char.head()

,Estimate!!HOUSEHOLDS BY TYPE!!Total households,Margin of Error!!HOUSEHOLDS BY TYPE!!Total households,Percent!!HOUSEHOLDS BY TYPE!!Total households,Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household,Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household,Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household!!With children of the householder under 18 years,Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household!!With children of the householder under 18 years,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household!!With children of the householder under 18 years,Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household!!With children of the householder under 18 years,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household,Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household,Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household,Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household!!With children of the householder under 18 years,Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household!!With children of the householder under 18 years,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household!!With children of the householder under 18 years,Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household!!With children of the householder under 18 years,"Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present","Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present","Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present","Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!With children of the householder under 18 years","Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!With children of the householder under 18 years","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!With children of the householder under 18 years","Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!With children of the householder under 18 years","Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone","Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone","Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone","Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over","Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over","Percent Margin of Error!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over","Estimate!!HOUSEHOLDS BY TYPE!!Total households!!Femal

In [107]:
perc_social_char = social_char[[i for i in social_char.columns if (i.startswith("Percent!!") & all(social_char[i] != "(X)"))] + ["Geographic Area Name"]]

In [108]:
perc_social_char["POSTAL_CODE"] = [i[6:] for i in perc_social_char['Geographic Area Name']]

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/1943255995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_social_char["POSTAL_CODE"] = [i[6:] for i in perc_social_char['Geographic Area Name']]


In [109]:
from uszipcode import SearchEngine, SimpleZipcode, ComprehensiveZipcode

In [110]:
# search = SearchEngine()
# zipcode = search.by_zipcode('90023')
# zipcode.county + ", " + zipcode.state_long

'Los Angeles County, California'

In [118]:
def state_fill(x:str) -> str:
    try:
        search = SearchEngine()
        state = search.by_zipcode(x).state
    except:
        state= ""
    return state

In [112]:
perc_social_char["STATE"] = perc_social_char["POSTAL_CODE"].apply(state_fill)

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/233638724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_social_char["STATE"] = perc_social_char["POSTAL_CODE"].apply(state_fill)


In [115]:
# perc_social_char.to_csv("../data/census/perc_social_char_DP02.csv")

#### DP03

In [117]:
econ_char = pd.read_csv("../data/ACSDP5Y2020.DP03_data_with_overlays_2022-03-17T153550.csv", header = 1,low_memory=False)

In [120]:
perc_econ_char = econ_char[[i for i in econ_char.columns if (i.startswith("Percent!!") & all(econ_char[i] != "(X)"))] + ["Geographic Area Name"]]
perc_econ_char["POSTAL_CODE"] = [i[6:] for i in perc_econ_char['Geographic Area Name']]
perc_econ_char["STATE"] = perc_econ_char["POSTAL_CODE"].apply(state_fill)

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/472093404.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_econ_char["POSTAL_CODE"] = [i[6:] for i in perc_econ_char['Geographic Area Name']]
/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/472093404.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_econ_char["STATE"] = perc_econ_char["POSTAL_CODE"].apply(state_fill)


In [123]:
# perc_econ_char.to_csv("../data/census/perc_econ_char_DP03.csv")

#### DP04

In [124]:
house_char = pd.read_csv("../data/ACSDP5Y2020.DP04_data_with_overlays_2022-03-17T150547.csv", header = 1, low_memory=False)

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/1605608971.py:1: DtypeWarning: Columns (296,297) have mixed types. Specify dtype option on import or set low_memory=False.
  house_char = pd.read_csv("../data/ACSDP5Y2020.DP04_data_with_overlays_2022-03-17T150547.csv", header = 1)


In [125]:
perc_house_char = house_char[[i for i in house_char.columns if (i.startswith("Percent!!") & all(house_char[i] != "(X)"))] + ["Geographic Area Name"]]
perc_house_char["POSTAL_CODE"] = [i[6:] for i in perc_house_char['Geographic Area Name']]
perc_house_char["STATE"] = perc_house_char["POSTAL_CODE"].apply(state_fill)

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/4283327304.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_house_char["POSTAL_CODE"] = [i[6:] for i in perc_house_char['Geographic Area Name']]
/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/4283327304.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_house_char["STATE"] = perc_house_char["POSTAL_CODE"].apply(state_fill)


In [126]:
# perc_house_char.to_csv("../data/census/perc_house_char_DP04.csv")

#### DP05

In [127]:
demo_house_esti = pd.read_csv("../data/ACSDP5Y2020.DP05_data_with_overlays_2022-03-17T153242.csv", header = 1,low_memory=False)

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/3779330382.py:1: DtypeWarning: Columns (1,5,9,17,29,33,37,41,45,57,73,81,93,97,101,105,113,117,121,129,249,277,281,301) have mixed types. Specify dtype option on import or set low_memory=False.
  demo_house_esti = pd.read_csv("../data/ACSDP5Y2020.DP05_data_with_overlays_2022-03-17T153242.csv", header = 1)


In [128]:
perc_demo_house_esti = demo_house_esti[[i for i in demo_house_esti.columns if (i.startswith("Percent!!") & all(demo_house_esti[i] != "(X)"))] + ["Geographic Area Name"]]
perc_demo_house_esti["POSTAL_CODE"] = [i[6:] for i in perc_demo_house_esti['Geographic Area Name']]
perc_demo_house_esti["STATE"] = perc_demo_house_esti["POSTAL_CODE"].apply(state_fill)

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/811986295.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_demo_house_esti["POSTAL_CODE"] = [i[6:] for i in perc_demo_house_esti['Geographic Area Name']]
/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/811986295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perc_demo_house_esti["STATE"] = perc_demo_house_esti["POSTAL_CODE"].apply(state_fill)


In [129]:
# perc_demo_house_esti.to_csv("../data/census/perc_demo_house_esti_DP05.csv")

In [131]:
# acs_overall_national = pd.concat([perc_social_char,perc_econ_char,perc_house_char,perc_demo_house_esti], join = "inner", axis = 1)

In [135]:
def merge_df(df_list):
    output_df = pd.DataFrame(df_list[0].drop(["Geographic Area Name","STATE"], axis = 1))
    for df in df_list[1:]:
        df = df.drop(["Geographic Area Name","STATE"], axis = 1)
        output_df = output_df.merge(df, how= "inner",  on="POSTAL_CODE")

    return output_df

In [136]:
acs_overall_national = merge_df([perc_social_char,perc_econ_char,perc_house_char,perc_demo_house_esti])

In [137]:
acs_overall_national.head()

,Percent!!HOUSEHOLDS BY TYPE!!Total households,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household,"Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone!!65 years and over",Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people under 18 years,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people 65 years and over,Percent!!RELATIONSHIP!!Population in households,Percent!!RELATIONSHIP!!Population in households!!Householder,Percent!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSHIP!!Population in households!!Unmarried partner,Percent!!RELATIONSHIP!!Population in households!!Child,Percent!!RELATIONSHIP!!Population in households!!Other relatives,Percent!!RELATIONSHIP!!Population in households!!Other nonrelatives,Percent!!MARITAL STATUS!!Males 15 years and over,Percent!!MARITAL STATUS!!Males 15 years and over!!Never married,"Percent!!MARITAL STATUS!!Males 15 years and over!!Now married, except separated",Percent!!MARITAL STATUS!!Males 15 years and over!!Separated,Percent!!MARITAL STATUS!!Males 15 years and over!!Widowed,Percent!!MARITAL STATUS!!Males 15 years and over!!Divorced,Percent!!MARITAL STATUS!!Females 15 years and over,Percent!!MARITAL STATUS!!Females 15 years and over!!Never married,"Percent!!MARITAL STATUS!!Females 15 years and over!!Now married, except separated",Percent!!MARITAL STATUS!!Females 15 years and over!!Separated,Percent!!MARITAL STATUS!!Females 15 years and over!!Widowed,Percent!!MARITAL STATUS!!Females 15 years and over!!Divorced,Percent!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months,"Percent!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Unmarried women (widowed, divorced, and never married)",Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Grandparents responsible for grandchildren,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!Less than 1 year,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!1 or 2 years,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!3 or 4 years,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!5 or more years,Percent!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years,Percent!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are female,Percent!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are married,Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school,"Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool",Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (g

In [138]:
acs_overall_national.insert(0, "POSTAL_CODE",acs_overall_national.pop("POSTAL_CODE"))

/var/folders/3j/z2qkp3593qx624tzwlfnllkr0000gn/T/ipykernel_5043/3022194990.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  acs_overall.insert(0, "POSTAL_CODE",acs_overall.pop("POSTAL_CODE"))


In [139]:
acs_overall_national.head()

,POSTAL_CODE,Percent!!HOUSEHOLDS BY TYPE!!Total households,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household,"Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone!!65 years and over",Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people under 18 years,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people 65 years and over,Percent!!RELATIONSHIP!!Population in households,Percent!!RELATIONSHIP!!Population in households!!Householder,Percent!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSHIP!!Population in households!!Unmarried partner,Percent!!RELATIONSHIP!!Population in households!!Child,Percent!!RELATIONSHIP!!Population in households!!Other relatives,Percent!!RELATIONSHIP!!Population in households!!Other nonrelatives,Percent!!MARITAL STATUS!!Males 15 years and over,Percent!!MARITAL STATUS!!Males 15 years and over!!Never married,"Percent!!MARITAL STATUS!!Males 15 years and over!!Now married, except separated",Percent!!MARITAL STATUS!!Males 15 years and over!!Separated,Percent!!MARITAL STATUS!!Males 15 years and over!!Widowed,Percent!!MARITAL STATUS!!Males 15 years and over!!Divorced,Percent!!MARITAL STATUS!!Females 15 years and over,Percent!!MARITAL STATUS!!Females 15 years and over!!Never married,"Percent!!MARITAL STATUS!!Females 15 years and over!!Now married, except separated",Percent!!MARITAL STATUS!!Females 15 years and over!!Separated,Percent!!MARITAL STATUS!!Females 15 years and over!!Widowed,Percent!!MARITAL STATUS!!Females 15 years and over!!Divorced,Percent!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months,"Percent!!FERTILITY!!Number of women 15 to 50 years old who had a birth in the past 12 months!!Unmarried women (widowed, divorced, and never married)",Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Grandparents responsible for grandchildren,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!Less than 1 year,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!1 or 2 years,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!3 or 4 years,Percent!!GRANDPARENTS!!Number of grandparents living with own grandchildren under 18 years!!Years responsible for grandchildren!!5 or more years,Percent!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years,Percent!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are female,Percent!!GRANDPARENTS!!Number of grandparents responsible for own grandchildren under 18 years!!Who are married,Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school,"Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Nursery school, preschool",Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Kindergarten,Percent!!SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementa

In [140]:
# acs_overall_national.to_csv("../data/census/acs_overall_national.csv")

In [142]:
cutoff_all_lead_acs = cutoff_all_lead.merge(acs_overall_national, how = "left", on = "POSTAL_CODE")

In [143]:
cutoff_all_lead_acs.head()

,index,ID,IS_DELETED,MASTER_RECORD_ID,LAST_NAME,FIRST_NAME,SALUTATION,MIDDLE_NAME,SUFFIX,NAME,RECORD_TYPE_ID,TITLE,COMPANY,STREET,CITY,STATE,POSTAL_CODE,COUNTRY,LATITUDE,LONGITUDE,GEOCODE_ACCURACY,PHONE,MOBILE_PHONE,FAX,EMAIL,WEBSITE,PHOTO_URL,DESCRIPTION,LEAD_SOURCE,STATUS,INDUSTRY,RATING,ANNUAL_REVENUE,NUMBER_OF_EMPLOYEES,OWNER_ID,HAS_OPTED_OUT_OF_EMAIL,IS_CONVERTED,CONVERTED_DATE,CONVERTED_ACCOUNT_ID,CONVERTED_CONTACT_ID,CONVERTED_OPPORTUNITY_ID,IS_UNREAD_BY_OWNER,CREATED_DATE,CREATED_BY_ID,LAST_MODIFIED_DATE,LAST_MODIFIED_BY_ID,SYSTEM_MODSTAMP,LAST_ACTIVITY_DATE,DO_NOT_CALL,HAS_OPTED_OUT_OF_FAX,LAST_VIEWED_DATE,LAST_REFERENCED_DATE,LAST_TRANSFER_DATE,JIGSAW,JIGSAW_CONTACT_ID,EMAIL_BOUNCED_REASON,EMAIL_BOUNCED_DATE,INDIVIDUAL_ID,POSTAL_CODE_C,MOST_RECENT_CAMPAIGN_C,ALTERNATE_EMAIL_C,APEX_PROCESSED_C,KNOCK_WEBHOOK_C,ALTERNATE_PHONE_C,TOTAL_SALES_C,BOOK_SIZE_SOW_PREFERRED_C,EMAIL_BOUNCED_C,EVENT_TYPE_C,IS_LEAD_C,AVERAGE_LIST_PRICE_C,FSBO_ADDRESS_C,FSBO_LISTING_PRICE_C,FIRST_TASK_DATE_C,FIRST_EVENT_DATE_C,BROKERAGE_1_C,TOTAL_TRANSACTION_COUNT_C,COUNTY_C,SALES_LOFT_1_MOST_RECENT_CADENCE_NAME_C,SALES_LOFT_1_MOST_RECENT_CADENCE_NEXT_STEP_DUE_DATE_C,SALES_LOFT_1_MOST_RECENT_LAST_COMPLETED_STEP_C,DISQUALIFIED_NOTES_C,FA_LOYAL_LEAD_C,LS_SALES_C,TOTAL_LISTING_COUNT_C,PREFERRED_PROVIDER_BY_LISTING_COUNT_C,PRODUCT_DEMO_DATE_C,PREFERRED_ESCROW_COMPANY_C,ROLE_C,APP_DOWNLOAD_DATE_C,PREFERRED_TITLE_COMPANY_C,AGENT_MLS_ID_C,ORDER_ENTRY_NOTES_C,LEVEL_VETTING_C,LICENSE_NO_C,PARDOT_REASSIGNMENT_C,AGENT_ROLE_C,LISTING_PRICE_C,TOTAL_MODUS_TRANSACTION_COUNT_C,DIGITAL_TE_USER_C,PI_NEEDS_SCORE_SYNCED_C,PI_PARDOT_LAST_SCORED_AT_C,PI_CAMPAIGN_C,PI_COMMENTS_C,PI_CONVERSION_DATE_C,PI_CONVERSION_OBJECT_NAME_C,PI_CONVERSION_OBJECT_TYPE_C,PI_CREATED_DATE_C,PI_FIRST_ACTIVITY_C,PI_FIRST_SEARCH_TERM_C,PI_FIRST_SEARCH_TYPE_C,PI_FIRST_TOUCH_URL_C,PI_GRADE_C,PI_LAST_ACTIVITY_C,PI_NOTES_C,PI_PARDOT_HARD_BOUNCED_C,PI_SCORE_C,PI_URL_C,PI_UTM_CAMPAIGN_C,PI_UTM_CONTENT_C,PI_UTM_MEDIUM_C,PI_UTM_SOURCE_C,PI_UTM_TERM_C,LAST_MODIFIED_C,PREFERRED_PLATFORM_C,AVERAGE_CLOSE_PRICE_C,AVERAGE_TRANSACTION_AMOUNT_C,BS_COUNT_C,DISQUALIFIED_REASONS_C,FA_DIRECTED_ESCROW_TRANSACTIONS_C,FA_DIRECTED_TRANSACTIONS_C,LEAD_NUMBER_C,LEAD_RANKING_C,MATRIX_UNIQUE_ID_C,MEAL_TRACKER_C,PMXAGENT_ID_C,PREFERRED_ESCROW_SOW_C,SECONDARY_ESCROW_CO_C,SECONDARY_TITLE_CO_C,BS_SALES_C,TOTAL_DIRECTED_ESCROW_TRANSACTIONS_C,CHECKBOX_C,NMLS_ID_C,REFERRER_C,PROPERTY_ADDRESS_C,REFERRAL_EMAIL_C,COMING_SOON_ADDRESS_C,KCA_C,_FIVETRAN_SYNCED,EMAIL_UNKNOWN_DELIVERABILITY_C,ACCOUNT_MANAGER_C,SALES_DEVELOPMENT_REP_C,RELATIONSHIP_MANAGER_C,WEB_LEAD_C,KNOCK_WEBHOOK_2021_C,DO_NOT_CONTACT_C,MOBILE_INDEXED_C,DEMO_SET_ON_C,DEMO_SCHEDULED_FOR_C,STATE_STD,LEADCONVERTED,Percent!!HOUSEHOLDS BY TYPE!!Total households,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married-couple household,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household,"Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse/partner present!!Householder living alone!!65 years and over","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone","Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse/partner present!!Householder living alone!!65 years and over",Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people under 18 years,Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with one or more people 65 years and over,Percent!!RELATIONSHIP!!Population in households,Percent!!RELATIONSHIP!!Population in households!!Householder,Percent!!RELATIONSHIP!!Population in households!!Spouse,Percent!!RELATIONSH

In [144]:
# cutoff_all_lead_acs.to_csv("../data/census/cutoff_all_lead_acs.csv")